In [11]:
import psycopg2
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from plotly.offline import plot 
import plotly.graph_objects as go
import warnings
import datetime
warnings.filterwarnings('ignore')

# import q_credentials.db_secmaster_cloud_cred as db_secmaster_cred
import q_credentials.db_secmaster_cred as db_secmaster_cred
import q_credentials.db_indicator_cred as db_indicator_cred


In [12]:
# connect to our securities_master database
conn_secmaster = psycopg2.connect(host=db_secmaster_cred.dbHost , database=db_secmaster_cred.dbName, user=db_secmaster_cred.dbUser, password=db_secmaster_cred.dbPWD)

In [13]:
conn_indicator = psycopg2.connect(host=db_indicator_cred.dbHost , database=db_indicator_cred.dbName, user=db_indicator_cred.dbUser, password=db_indicator_cred.dbPWD)

In [14]:
# sql="""SELECT s.ticker, max(d.symbol_id), max(d.date_price)
# FROM public.symbol s inner join d_data d on s.id= d.symbol_id
# group by s.ticker"""
# df_ticker_last_day=pd.read_sql(sql,con=conn_secmaster)

In [15]:
# df_ticker_last_day

In [16]:
# sql="""select date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from daily_data where stock_id=71"""
# df_stock=pd.read_sql(sql,con=conn_secmaster)

In [17]:
# import datetime

In [18]:
sql="select * from indicator"
ind_list=list(pd.read_sql(sql,con=conn_indicator)['name'])
ind_list

['candle_1', 'candle_2', 'candle_3', 'anomaly', 'level']

In [19]:
symbol_id='EUR_USD'#'AAPL'#'BOM500114'
start_date=datetime.datetime(2018,1,1).strftime("%Y-%m-%d")
indicator_name = 'candle_1'

In [10]:
# sql="select d.date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from d_data d join symbol s on d.symbol_id = s.id where s.ticker='%s' and d.date_price > '%s'" %(symbol_id, start_date)
# df_price=pd.read_sql(sql,con=conn_secmaster)

In [11]:
# sql="select d.date_price as date, d.value from d_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s'" %(symbol_id, indicator_name, start_date)
# df_indicator=pd.read_sql(sql,con=conn_indicator)

In [12]:
# df_indicator.set_index('date',inplace=True)

In [13]:
# df_indicator.head()

,value
date,
2018-01-02,"{'level': 0.0, 'pattern_name': ''}"
2018-01-03,"{'level': 0.0, 'pattern_name': ''}"
2018-01-04,"{'level': 0.0, 'pattern_name': ''}"
2018-01-05,"{'level': 0.0, 'pattern_name': ''}"
2018-01-08,"{'level': 0.0, 'pattern_name': ''}"


In [46]:
df_all_ind=pd.DataFrame()
for ind in ind_list:
    print(ind)
    sql="select d.date_price as date, d.value from w_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s'" %(symbol_id, ind, start_date)
    df_indicator=pd.read_sql(sql,con=conn_indicator)
    df_indicator.set_index('date',inplace=True)
    df_indicator=pd.concat([df_indicator.drop(['value'], axis=1), df_indicator['value'].apply(pd.Series)], axis=1)
    df_indicator.columns=[ind+"_"+col for col in df_indicator.columns]
    if df_all_ind.empty:
#         df_all_ind=pd.merge(left=df_price, right=df_indicator,on='date')
        df_all_ind=df_indicator
    else:
        df_all_ind=pd.merge(left=df_all_ind, right=df_indicator,on='date')

candle_1
candle_2
candle_3
anomaly
level


In [47]:
df_all_ind.head()

,candle_1_level,candle_1_pattern_name,candle_2_level,candle_2_pattern_name,candle_3_level,candle_3_pattern_name,anomaly_bear,anomaly_body_size,anomaly_body_size_dev,anomaly_body_size_ema,anomaly_bull,anomaly_close,anomaly_gap,anomaly_gap_dev,anomaly_gap_ema,anomaly_gap_signal,anomaly_high,anomaly_low,anomaly_open,anomaly_vol_anomaly,anomaly_vol_anomaly_2,anomaly_vol_dev,anomaly_vol_ema,anomaly_volume,anomaly_wick_bear,anomaly_wick_bear_dev,anomaly_wick_bear_ema,anomaly_wick_bear_signal,anomaly_wick_bull,anomaly_wick_bull_dev,anomaly_wick_bull_ema,anomaly_wick_bull_signal,level_close,level_high,level_level,level_low,level_nearest_resistance_atr_range,level_nearest_resistance_duration,level_nearest_resistance_range,level_nearest_support_atr_range,level_nearest_support_duration,level_nearest_support_range,level_open,level_resistance,level_support
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-05,-10.0,short,0.0,,0.0,,0.0,0.00226,0.703838,0.007631,1.0,1.20297,0.00032,-0.641515,0.000893,0.000000,1.20890,1.20010,1.20071,0.545937,0.0,0.224343,178750.341002,138649.0,0.00061,-0.812972,0.003262,0.000000,0.00593,0.288792,0.004601,0.288792,1.20297,1.20890,0.0,1.20010,0.222004,119.0,"[1.20928, 1.20353]",0.562281,35,"[1.19242, 1.19612]",1.20071,"[[1.20928, 1.20353, 2017-09-08 00:00:00, True]]","[[1.19242, 1.19612, 2017-12-01 00:00:00, False..."
2018-01-12,0.0,,0.0,,0.0,,0.0,0.01697,0.819067,0.009329,1.0,1.21968,-0.00026,-0.665646,0.000778,-0.000000,1.22193,1.19159,1.20271,0.599394,0.0,0.268199,190079.006275,241058.0,0.01112,1.370824,0.004690,1.370824,0.00225,-0.460912,0.004174,0.000000,1.21968,1.22193,0.0,1.19159,NaN,NaN,NaN,0.719663,126,"[1.20353, 1.20928]",1.20271,NaN,"[[1.20353, 1.20928, 2017-09-08 00:00:00, False..."
2018-01-19,5.0,DOJI_SPINOFF,0.0,,0.0,,0.0,0.00276,0.660709,0.008135,1.0,1.22211,-0.00033,-0.526021,0.000696,-0.000000,1.23230,1.21653,1.21935,0.318165,0.0,0.518449,214829.732407,326208.0,0.00282,-0.351767,0.004350,0.000000,0.01019,0.934472,0.005268,0.934472,1.22211,1.23230,0.0,1.21653,NaN,NaN,NaN,0.876840,133,"[1.20353, 1.20928]",1.21935,NaN,"[[1.20353, 1.20928, 2017-09-08 00:00:00, False..."
2018-01-26,0.0,,0.0,,0.0,,0.0,0.01580,0.658216,0.009528,1.0,1.24296,0.00505,2.394212,0.001488,2.394212,1.25374,1.22141,1.22716,0.143628,0.0,0.781793,259999.962878,463266.0,0.00575,0.248702,0.004605,0.248702,0.01078,0.719341,0.006270,0.719341,1.24296,1.25374,0.0,1.22141,NaN,NaN,NaN,1.757897,140,"[1.20353, 1.20928]",1.22716,NaN,"[[1.20353, 1.20928, 2017-09-08 00:00:00, False..."
2018-02-02,5.0,DOJI_SPINOFF,0.0,,0.0,,0.0,0.00286,0.656080,0.008316,1.0,1.24566,-0.00016,-0.871631,0.001246,-0.000000,1.25233,1.23349,1.24280,0.328454,0.0,0.499369,292453.787809,438496.0,0.00931,0.705042,0.005460,0.705042,0.00667,0.051619,0.006343,0.051619,1.24566,1.25233,0.0,1.23349,NaN,NaN,NaN,1.923908,147,"[1.20353, 1.20928]",1.24280,NaN,"[[1.20353, 1.20928, 2017-09-08 00:00:00, False..."


In [50]:
df_all_ind.rename(columns={'anomaly_close':'close','anomaly_low':'low','anomaly_high':'high','anomaly_open':'open','anomaly_volume':'volume'},inplace=True)

In [51]:
df_all_ind.columns

Index(['candle_1_level', 'candle_1_pattern_name', 'candle_2_level',
       'candle_2_pattern_name', 'candle_3_level', 'candle_3_pattern_name',
       'anomaly_bear', 'anomaly_body_size', 'anomaly_body_size_dev',
       'anomaly_body_size_ema', 'anomaly_bull', 'close', 'anomaly_gap',
       'anomaly_gap_dev', 'anomaly_gap_ema', 'anomaly_gap_signal', 'high',
       'low', 'open', 'anomaly_vol_anomaly', 'anomaly_vol_anomaly_2',
       'anomaly_vol_dev', 'anomaly_vol_ema', 'volume', 'anomaly_wick_bear',
       'anomaly_wick_bear_dev', 'anomaly_wick_bear_ema',
       'anomaly_wick_bear_signal', 'anomaly_wick_bull',
       'anomaly_wick_bull_dev', 'anomaly_wick_bull_ema',
       'anomaly_wick_bull_signal', 'level_close', 'level_high', 'level_level',
       'level_low', 'level_nearest_resistance_atr_range',
       'level_nearest_resistance_duration', 'level_nearest_resistance_range',
       'level_nearest_support_atr_range', 'level_nearest_support_duration',
       'level_nearest_support_rang

In [52]:
pd.options.display.max_columns = None

In [53]:
def level_plot(df):
    support_ls = [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_support']]
    resistance_ls = [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_resistance']]
    end_dt=df.index[-1]
    res_plot_ls=[]
    sup_plot_ls=[]
    for res in resistance_ls[:5]:
        res_plot_ls.append(dict(x0=res[2],x1=end_dt,y0=res[0],y1=res[1],yref='y1',opacity=.2,fillcolor='Red',line=dict(color="black",width=1)))
    for sup in support_ls[:5]:
        sup_plot_ls.append(dict(x0=sup[2],x1=end_dt,y0=sup[0],y1=sup[1],yref='y1',opacity=.2,fillcolor='green',line=dict(color="black",width=1)))
    return (res_plot_ls+sup_plot_ls)

In [54]:
# df.columns

In [55]:
# candle1_plot_ls=[]
# for i,row in df.tail(10).iterrows():
#     if(row['candle_1_pattern_name']):
# #         print(row['candle_1_pattern_name'])
#         print(pd.to_datetime(i), (i+datetime.timedelta(days=1)))
#         candle1_plot_ls.append(dict(x0=i,x1=(i+datetime.timedelta(days=2)),y0=row['high'],y1=row['low'],yref='y1',opacity=.2,fillcolor='blue',line=dict(color="black",width=1)))

In [56]:
# df=df_all_ind.set_index('date')
df=df_all_ind
interested_feature='anomaly_vol_anomaly'

In [57]:
df_candle_1=df[-10:]
df_candle_1=df_candle_1[df_candle_1['candle_1_pattern_name']!='']
df_candle_1['pattern']='1'

df_candle_2=df[-10:]
df_candle_2=df_candle_2[df_candle_2['candle_2_pattern_name']!='']
df_candle_2['pattern']='2'

df_candle_3=df[-10:]
df_candle_3=df_candle_3[df_candle_3['candle_3_pattern_name']!='']
df_candle_3['pattern']='3'

In [58]:
# Plotting
data = [ dict(
    type = 'candlestick',
    open = df.open,
    high = df.high,
    low = df.low,
    close = df.close,
    x = df.index,
    yaxis = 'y1',
    name = 'price'
)]


data.append( dict( x=df.index, y=df.volume,                         
                         marker=dict( color='blue' ),
                         type='bar', yaxis='y2', name='Volume'))

data.append( dict( x=df.index, y=df[interested_feature],                         
                         marker=dict( color='red' ),
                         type='scatter', yaxis='y3', name=interested_feature))

data.append( dict( x=df_candle_1.index, y=df_candle_1['high'],
                         text=df_candle_1['pattern'],
                         mode="markers+text",textfont_size=15,textposition="top center",
                         marker=dict( color='blue' ),
                         type='scatter', yaxis='y1', name='candle_patter_1'))

data.append( dict( x=df_candle_2.index, y=df_candle_2['high'],
                         text=df_candle_2['pattern'],
                         mode="markers+text",textfont_size=15,textposition="top center",
                         marker=dict( color='red' ),
                         type='scatter', yaxis='y1', name='candle_patter_2'))


data.append( dict( x=df_candle_3.index, y=df_candle_3['high'],
                         text=df_candle_3['pattern'],
                         mode="markers+text",textfont_size=15,textposition="top center",
                         marker=dict( color='red' ),
                         type='scatter', yaxis='y1', name='candle_patter_3'))


layout=dict()    
layout['xaxis'] = dict( rangeslider = dict( visible = False ),autorange=True,fixedrange=False,visible=False,type='category')#type='category',
layout['yaxis'] = dict( domain = [0.2, 1],autorange = True,fixedrange=False)
layout['yaxis2'] = dict( domain = [0.0, 0.1],autorange = True,fixedrange=False)
layout['yaxis3'] = dict( domain = [0.1, 0.2],autorange = True,fixedrange=False)
# layout['shapes'] = [dict(x0=start_dt, x1=end_dt, y0=3057.9, y1=3060.85, yref='y1',opacity=.2,line=dict(color="Red",width=1))]#fillcolor='green',line_width=2
layout['shapes'] = level_plot(df)
# layout['shapes'] = candle1_plot_ls

# df.to_csv("stock_price_indicator.csv",index=True)

fig = dict( data=data, layout=layout )

In [59]:
figure=go.FigureWidget(data=data, layout=layout)

In [60]:
def zoom(layout, x_range):
    print(x_range)
    in_view = df.iloc[int(figure.layout.xaxis.range[0]):int(figure.layout.xaxis.range[1])]
    figure.layout.yaxis2.range = [in_view.volume.min(), in_view.volume.max()]
    figure.layout.yaxis3.range = [in_view[interested_feature].min(), in_view[interested_feature].max()]

In [61]:
figure.layout.on_change(zoom, 'xaxis.range')

In [62]:
figure.update_layout(margin=dict(l=20, r=10),paper_bgcolor="LightSteelBlue",width=2200, height=1000)

FigureWidget({
    'data': [{'close': array([1.20297, 1.21968, 1.22211, 1.24296, 1.24566, 1.22512, 1.24097, 1.…